In [13]:
import numpy as np
import h5py
import pandas as pd
import collections

In [2]:
f = h5py.File('../track.h5', 'r')
list(f.keys())
df = pd.read_hdf('../track.h5','tracks')

In [3]:
df

,index,jet_phi_orig,jet_eta_orig,jet_pt_orig,jet_trk_chi2,jet_trk_ndf,jet_trk_ip3d_d0,jet_trk_ip3d_d02D,jet_trk_ip3d_z0,jet_trk_ip3d_d0sig,...,jet_trk_nsplitBLHits,jet_trk_nPixHits,jet_trk_nPixHoles,jet_trk_nsharedPixHits,jet_trk_nsplitPixHits,jet_trk_nSCTHits,jet_trk_nSCTHoles,jet_trk_nsharedSCTHits,jet_trk_pt,jet_trk_eta
0,0,2.970113,2.223017,85018.460938,28.354776,19.0,-0.064755,-0.064755,-0.052846,-1.968600,...,0,7,0,0,0,10,0,0,5186.198730,2.159084
1,1,2.970113,2.223017,85018.460938,13.942179,15.0,0.025097,0.025097,0.190301,1.812325,...,0,6,0,0,0,8,0,0,37920.195312,2.202172
2,2,2.970113,2.223017,85018.460938,9.871879,15.0,0.275291,0.275291,0.139770,9.434825,...,0,5,0,0,0,10,0,0,8602.825195,2.212207
3,3,2.970113,2.223017,85018.460938,12.308242,13.0,1.080500,1.080500,0.649810,14.998597,...,0,5,0,0,0,8,0,0,2081.342773,2.297454
4,4,-0.937904,0.102383,58043.121094,41.761860,45.0,0.084461,0.084461,0.167722,4.083795,...,0,5,0,0,0,8,0,0,6849.432617,0.126085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3546877,3546877,1.097530,-1.923653,23333.771484,13.093798,6.0,7.244178,7.244178,31.740755,27.117729,...,0,2,1,0,0,7,0,1,547.481079,-1.963377
3546878,3546878,1.097530,-1.923653,23333.771484,44.229511,24.0,2.539800,2.539800,17.357218,6.958940,...,0,3,0,0,0,5,2,1,738.593018,-1.979410
3546879,3546879,1.097530,-1.923653,23333.771484,30.325817,21.0,6.043699,-6.043699,30.994108,52.438911,...,0,1,0,0,0,6,1,0,5292.955078,-1.986661
3546880,3546880,1.097530,-1.923653,23333.771484,5.400502,5.0,-5.732069,-5.732069,-33.762501,-16.135002,...,0,3,0,0,0,4,2,0,635.220825,-1.732470


In [8]:
def configurePlot(jet, text):
    fig = plt.figure(figsize=(8, 5.5), constrained_layout=True)
    gs = gridspec.GridSpec(8, 1, figure=fig)
    ax1 = fig.add_subplot(gs[:5, 0])
    ax2 = fig.add_subplot(gs[5:, 0], sharex=ax1)

    ax2.set_xlabel('$b$-efficiency', fontsize=12, horizontalalignment='right', x=1.0)
    if jet == "c":
        ax1.set_ylabel('$c$-jet rejection', fontsize=12, horizontalalignment='right', y=1.0)
    if jet == "u":
        ax1.set_ylabel('light flavour jet rejection', fontsize=12, horizontalalignment='right', y=1.0)
    ax1.set_yscale('log')
    ax2.set_ylabel('Ratio', fontsize=12)
    ax2.grid()
    ax1.grid()
    plt.setp(ax1.get_xticklabels(), visible=False)
    
    if text is not None:
        ax1.text(**text, transform=ax1.transAxes)

    # Set the axes to be the same as those used in the pub note
    ax1.set_xlim(xmin, 1)
    ax2.set_xlim(xmin, 1)
    ax1.set_ylim(1, ymax)
        
    return ax1, ax2

In [10]:
def plotROCRatio(evalDicts, outDir, binominalErrors=True, nTest=1960184, text=None,
                 legFontSize=7.5, legcols=1):
    figname = PFlow_ROC
    
    # Check if suitable for binominal errors
    if binomialErrors and nTest == 0:
        print("Error: Requested binomialErrors, but did not pass nTest.",
              "Will NOT plot rej errors.")
        binomialErrors = False
    if type(nTest) != list:
        nTest = [nTest] * len(teffs)
    xx = np.linspace(xmin, 1, 101)

    ax1, ax2 = configurePlot(ylabel, text)
    x = evalDicts["beffs"]
    for jet in ['c', 'u']:
        figname += '_' + jet + 'rej'
        ax1, ax2 = configurePlot(jet, text)
        
        for model in models:
             y = evalDicts[model + '_{}rej'.format(jet)]
            # sort out color and label here or in keywords
        
            ax1.plot(x, y, color=color,label[model], linewidth=2)
            if binomialErrors:
                yerr = binominalErrors(y)

            # Approximate function using x and y then interpolate other x values using f
            f = pchip(x, y)
            if model == 'nom':
                f0 = f
                ratio_i = np.ones(len(xx))
                ratio_ix = 1
            else:
                ratio_i = f(xx) / f0(xx)
                ratio_ix = f(x) / f0(x)
                print (ratio_ix) #check if the above is needed
                print (ratio_i)

            ax2.plot(xx, ratio_i, color=color, linewidth=1.6)
            if binomialErrors:
                ax2.fill_between(x, ratio_ix - yerr/f(x), ratio_ix + yerr / f(x), color=color, 
                                 alpha=0.5, zorder=1) # check if f(x) = y
    
        ax1.legend(loc='best', fontsize=legFontSize, ncol=legcols) # check legend works without
        #fake lines
        plt.savefig(outDir + "/" + figName + ".pdf", bbox_inches='tight', pad_inches=0.04)
        

In [16]:
def getRejsValues(models, algo='', sample=''):
    evalDicts = {}
    # Read in file, get crejs and lrejs for a given algo
    for model in models:
        df = pd.read_hdf(modelFilename, sample)
        for jet in ['c', 'u']:
            evalDicts[model + '_{}rej'.format(jet)] = df['{}_{}rej'.format(algo, jet)]
        if model == 'nom':
            evalDicts["beffs"] = df["beffs"]
    
    return evalDicts

In [17]:
def binominalErrors(y):
    yerr = np.power(y, 2) * eff_err(np.divide(1, y), nte)
    y1 = y - yerr
    y2 = y + yerr
    
    ax1.fill_between(x, y1, y2, color=color, zorder=2)
    
    return yerr